<a href="https://colab.research.google.com/github/iskra3138/colab_repo/blob/master/XAI_Visualization_Methods_TF1_x_Share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

copy a trained model from GCP

In [0]:
!gsutil cp gs://iskra3138_mvtec_tfrecords/my_mvtec_tpumodel.h5 ./

In [0]:
model_file = 'my_mvtec_tpumodel.h5'
activation_layer = 'conv5_block3_out'

In [0]:
%tensorflow_version 1.x

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from PIL import Image as Image
import cv2

import glob

import tensorflow as tf
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops


import scipy
from scipy import ndimage
from skimage.measure import label, regionprops

%matplotlib inline

# import model

In [0]:
model = tf.keras.models.load_model(model_file)

# Make Dataset

In [0]:
#@title TFRecord Parsing Functions [Run Me!!!!]
import os

AUTO = tf.data.experimental.AUTOTUNE

IMAGE_SIZE =  [224, 224]

validation_fns = 'gs://iskra3138_mvtec_tfrecords/valid.tfrecords'

def parse_tfrecord(example):
    features = {
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'depth': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    label = example['label']
    #label = tf.one_hot(indices=label, depth=2)
    image = tf.io.decode_jpeg(example['image_raw'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, IMAGE_SIZE)
    
    return image, label

def load_dataset(filenames):
  # Read from TFRecords. For optimal performance, we interleave reads from multiple files.
  records = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  return records.map(parse_tfrecord, num_parallel_calls=AUTO)

def NG_filter_fn(img, label):
  return tf.math.equal(label, 0)
def OK_filter_fn(img, label):
  return tf.math.equal(label, 1)

def make_class_sapling (fns, class_idx, batch_size) :
  if class_idx == 0 :
    NG_dataset = load_dataset(fns).filter(NG_filter_fn).shuffle(1000).batch(batch_size).prefetch(AUTO).repeat()
    NG_iter = NG_dataset.make_one_shot_iterator()
    return NG_iter
  else :
    OK_dataset = load_dataset(fns).filter(OK_filter_fn).shuffle(1000).batch(batch_size).prefetch(AUTO).repeat()
    OK_iter = OK_dataset.make_one_shot_iterator()
    return OK_iter
    

# Import GradCAM & Guided GradCAM

In [0]:
#@title import Util Code [Run Me!!]
def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if tf.keras.backend.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def generate_bbox(img, cam, threshold):
    labeled, nr_objects = ndimage.label(cam > threshold)
    props = regionprops(labeled)
    return props

In [0]:
#@title import Grad-CAM Code [Run Me!!]
class GradCAM:
  def __init__(self, model, activation_layer):
    self.model = model
    self.activation_layer = activation_layer
    self.gradient_fn = self._get_gradcam_tensor_function()

  # get partial tensor graph of CNN model
  def _get_gradcam_tensor_function(self):
    model_input = self.model.input
    class_idx = tf.argmax(model.output[0])
    
    y_c = self.model.outputs[0].op.inputs[0][0, class_idx]
    A_k = self.model.get_layer(self.activation_layer).output
    
    grads = K.gradients(y_c, A_k)[0]
    gradient_fn = K.function([model.input], [A_k, grads, model.output])
    return gradient_fn

  # generate Grad-CAM
  def generate(self, input_tensor):
    conv_output, grad_val, predictions = self.gradient_fn([input_tensor])
    conv_output, grad_val = conv_output[0], grad_val[0]
    
    weights = np.mean(grad_val, axis=(0, 1))
    gradcam = np.dot(conv_output, weights)
    
    gradcam = cv2.resize(gradcam, (224, 224))
    
    ## Relu
    gradcam = np.maximum(gradcam, 0)
    return gradcam, predictions[0]

In [0]:
#@title import Guided Grad-CAM Code [Run Me!!]

class GuidedGradCam:
    def __init__(self, model, activation_layer, method='GuidedBackProp'):
        self.model = model
        #self.model_func = model_func
        self.activation_layer = activation_layer

        if method == 'BackProp':
            self._register_backprop_gradient()
            self.guided_model = self._modify_graph('BackProp')
        elif method == 'DeconvNet':
            self._register_deconvnet_gradient()
            self.guided_model = self._modify_graph('DeconvNet')
        elif method == 'GuidedBackProp':
            self._register_guidedbackprop_gradient()
            self.guided_model = self._modify_graph('GuidedBackProp')
        else:
            sys.exit('method must be (BackProp, DeconvNet, GuidedBackProp)')

        self.tensor_function = self.get_tensor_function()

    # register gradient
    def _register_backprop_gradient(self):
        if "BackProp" not in ops._gradient_registry._registry:
            @ops.RegisterGradient("BackProp")
            def _BackProp(op, grad):
                dtype = op.inputs[0].dtype
                return grad * tf.cast(op.inputs[0] > 0., dtype)

    def _register_deconvnet_gradient(self):
        if "DeconvNet" not in ops._gradient_registry._registry:
            @ops.RegisterGradient("DeconvNet")
            def _DeconvNet(op, grad):
                dtype = op.inputs[0].dtype
                return grad * tf.cast(grad > 0., dtype)

    def _register_guidedbackprop_gradient(self):
        if "GuidedBackProp" not in ops._gradient_registry._registry:
            @ops.RegisterGradient("GuidedBackProp")
            def _GuidedBackProp(op, grad):
                dtype = op.inputs[0].dtype
                return grad * tf.cast(grad > 0., dtype) * \
                       tf.cast(op.inputs[0] > 0., dtype)

    # modify model graph
    def _modify_graph(self, name):
        g = tf.get_default_graph()
        with g.gradient_override_map({'Relu': name}):
            new_model = tf.keras.models.load_model('my_mvtec_tpumodel.h5')
        return new_model

    # get partial tensor graph of CNN model
    def get_tensor_function(self, method='max', channel=0):
        model_input = self.guided_model.input
        layer_output = self.guided_model.get_layer(self.activation_layer).output

        '''if method == 'max':
            output = K.max(layer_output, axis=3)
        elif method == 'one':
            output = layer_output[:, :, :, channel]
        else:
            sys.exit('method must be (max, one)')'''

        tensor_function = K.function([model_input], [K.gradients(layer_output, model_input)[0]])
        return tensor_function

    # generate saliency map(gradient)
    def generate(self, input_tensor, gradcam):
        saliency = self.tensor_function([input_tensor])[0]
        guided_grad_cam = saliency * gradcam[...,np.newaxis]

        return guided_grad_cam[0]

### gradcam, guided_gradcma class 객체 선언

In [0]:
gradcam_gen = GradCAM(model, activation_layer)
guided_gradcam_gen = GuidedGradCam(model, activation_layer)

### Visualization

Make Batch Data randomly.

In [0]:
sample_size = 5 # viualization할 이미지 개수
class_idx = 0 # NG 이면 0, OK 이면 1

iter = make_class_sapling(validation_fns, class_idx = class_idx, batch_size = sample_size)
images, labels = iter.get_next()
with tf.Session() as sess:
  imgs = sess.run(images)

Make gradcam, guided_gradcam

In [0]:
gradcams=[]
guided_gradcams=[]

for i, img in enumerate(imgs):
    
    img_tensor = np.expand_dims(img, axis=0)
    
    gradcam, pred = gradcam_gen.generate(img_tensor)
    gradcams.append(gradcam)
    
    guided_gradcam = guided_gradcam_gen.generate(img_tensor, gradcam)
    guided_gradcams.append(guided_gradcam)

In [0]:
fig, axes = plt.subplots(sample_size, 4, figsize=(10, 10))
for i, img in enumerate(imgs):

    img =img
    props = generate_bbox(img, gradcams[i], 0.5) # (원본이미지, gradcam값, BBOX 작성을 위한 threshold)
    
    axes[i, 0].imshow(img)
    axes[i, 1].imshow(img)
    axes[i, 1].imshow(gradcams[i], cmap='jet', alpha=0.5)
    axes[i, 2].imshow(img)
    for b in props:
        bbox = b.bbox
        xs = bbox[1]
        ys = bbox[0]
        w = bbox[3] - bbox[1]
        h = bbox[2] - bbox[0]

        rect = patches.Rectangle((xs, ys), w, h, linewidth=2, edgecolor='r', facecolor='none')
        axes[i, 2].add_patch(rect)
        
    axes[i, 3].imshow(deprocess_image(guided_gradcams[i]))
    
    
    axes[i, 0].axis('off')
    axes[i, 1].axis('off')
    axes[i, 2].axis('off')
    axes[i, 3].axis('off')
    
    axes[0, 0].set_title("image", fontsize=18)
    axes[0, 1].set_title("Grad-CAM", fontsize=18)
    axes[0, 2].set_title("BBOX", fontsize=18)
    axes[0, 3].set_title("Guided-Grad-CAM", fontsize=18)
    
plt.tight_layout()
plt.show()

# IntegratedGradients

출처 ; https://github.com/hiranumn/IntegratedGradients/blob/master/examples/VGG%20example.ipynb

In [0]:
#@title IntegratedGradients Class import [Run Me!!!!]
################################################################
# Implemented by Naozumi Hiranuma (hiranumn@uw.edu)            #
#                                                              #
# Keras-compatible implmentation of Integrated Gradients       # 
# proposed in "Axiomatic attribution for deep neuron networks" #
# (https://arxiv.org/abs/1703.01365).                          #
#                                                              #
# Keywords: Shapley values, interpretable machine learning     #
################################################################

from __future__ import division, print_function
import numpy as np
from time import sleep
import sys
import keras.backend as K

from keras.models import Model, Sequential

'''
Integrated gradients approximates Shapley values by integrating partial
gradients with respect to input features from reference input to the
actual input. The following class implements the paper "Axiomatic attribution
for deep neuron networks".
'''
class integrated_gradients:
    # model: Keras model that you wish to explain.
    # outchannels: In case the model are multi tasking, you can specify which output you want explain .
    def __init__(self, model, outchannels=[], verbose=1):
    
        #get backend info (either tensorflow or theano)
        self.backend = K.backend()
        
        #load model supports keras.Model and keras.Sequential
        if isinstance(model, Sequential) or isinstance(model, tf.keras.Sequential)  :
            self.model = model.model
            
        elif isinstance(model, Model) or isinstance(model, tf.keras.Model):
            self.model = model
            
        else:
            print("Invalid input model")
            #return -1
        
        #load input tensors
        self.input_tensors = []
        for i in self.model.inputs:
            self.input_tensors.append(i)
        # The learning phase flag is a bool tensor (0 = test, 1 = train)
        # to be passed as input to any Keras function that uses 
        # a different behavior at train time and test time.
        self.input_tensors.append(K.learning_phase())
        
        #If outputchanels are specified, use it.
        #Otherwise evalueate all outputs.
        self.outchannels = outchannels
        if len(self.outchannels) == 0: 
            if verbose: print("Evaluated output channel (0-based index): All")
            if K.backend() == "tensorflow":
                self.outchannels = range(self.model.output.shape[1]._value)
            elif K.backend() == "theano":
                self.outchannels = range(self.model.output._keras_shape[1])
        else:
            if verbose: 
                print("Evaluated output channels (0-based index):")
                print(','.join([str(i) for i in self.outchannels]))
                
        #Build gradient functions for desired output channels.
        self.get_gradients = {}
        if verbose: print("Building gradient functions")
        
        # Evaluate over all requested channels.
        for c in self.outchannels:
            # Get tensor that calculates gradient
            #gradients = self.model.optimizer.get_gradients(self.model.output[:, c], self.model.input)
            gradients = tf.gradients(self.model.output[:, c], self.model.input)[0]

            # Build computational graph that computes the tensors given inputs
            self.get_gradients[c] = tf.keras.backend.function([self.input_tensors], [gradients])
            
            # This takes a lot of time for a big model with many tasks.
            # So lets print the progress.
            if verbose:
                sys.stdout.write('\r')
                sys.stdout.write("Progress: "+str(int((c+1)*1.0/len(self.outchannels)*1000)*1.0/10)+"%")
                sys.stdout.flush()
        # Done
        if verbose: print("\nDone.")
            
                
    '''
    Input: sample to explain, channel to explain
    Optional inputs:
        - reference: reference values (defaulted to 0s).
        - steps: # steps from reference values to the actual sample (defualted to 50).
    Output: list of numpy arrays to integrated over.
    '''
    def explain(self, sample, outc=0, reference=False, num_steps=50, verbose=0):
        
        # Each element for each input stream.
        samples = []
        numsteps = []
        step_sizes = []
        
        # If multiple inputs are present, feed them as list of np arrays. 
        if isinstance(sample, list):
            #If reference is present, reference and sample size need to be equal.
            if reference != False: 
                assert len(sample) == len(reference)
            for i in range(len(sample)):
                if reference == False:
                    _output = integrated_gradients.linearly_interpolate(sample[i], False, num_steps)
                else:
                    _output = integrated_gradients.linearly_interpolate(sample[i], reference[i], num_steps)
                samples.append(_output[0])
                numsteps.append(_output[1])
                step_sizes.append(_output[2])
        
        # Or you can feed just a single numpy arrray. 
        elif isinstance(sample, np.ndarray):
            _output = integrated_gradients.linearly_interpolate(sample, reference, num_steps)
            samples.append(_output[0])
            numsteps.append(_output[1])
            step_sizes.append(_output[2])
            
        # Desired channel must be in the list of outputchannels
        assert outc in self.outchannels
        if verbose: print("Explaning the "+str(self.outchannels[outc])+"th output.")
            
        # For tensorflow backend
        _input = []
        for s in samples:
            _input.append(s)
        _input.append(0)
        
        gradients = self.get_gradients[outc](_input)
        
        
        explanation = []
        for i in range(len(gradients)):
            _temp = np.sum(gradients[i], axis=0)
            explanation.append(np.multiply(_temp, step_sizes[i]))
           
        # Format the return values according to the input sample.
        if isinstance(sample, list):
            return explanation
        elif isinstance(sample, np.ndarray):
            return explanation[0]
        return -1

    
    '''
    Input: numpy array of a sample
    Optional inputs:
        - reference: reference values (defaulted to 0s).
        - steps: # steps from reference values to the actual sample.
    Output: list of numpy arrays to integrate over.
    '''
    @staticmethod
    def linearly_interpolate(sample, reference=False, num_steps=50):
        # Use default reference values if reference is not specified
        if reference is False: reference = np.zeros(sample.shape);

        # Reference and sample shape needs to match exactly
        assert sample.shape == reference.shape

        # Calcuated stepwise difference from reference to the actual sample.
        ret = np.zeros(tuple([num_steps] +[i for i in sample.shape]))
        for s in range(num_steps):
            ret[s] = reference+(sample-reference)*(s*1.0/num_steps)

        return ret, num_steps, (sample-reference)*(1.0/num_steps)


In [0]:
#@title VisualizationLibrary.visualization_lib [Run Me!!!!]
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image

from io import StringIO
from io import BytesIO
from IPython.display import display
from IPython.display import Image
from scipy import ndimage


def ConvertToGrayscale(attributions):
  return np.average(attributions, axis=2)


def Polarity(attributions, polarity):
  if polarity == 'positive':
    return np.clip(attributions, 0, 1)
  elif polarity == 'negative':
    return np.clip(attributions, -1, 0)
  else:
    raise ValueError('Unrecognized polarity option.')


def ComputeThresholdByTopPercentage(attributions,
                                    percentage=60,
                                    plot_distribution=True):
  """Compute the threshold value that maps to the top percentage of values.

  This function takes the cumulative sum of attributions and computes the set
  of top attributions that contribute to the given percentage of the total sum.
  The lowest value of this given set is returned.

  Args:
    attributions: (numpy.array) The provided attributions.
    percentage: (float) Specified percentage by which to threshold.
    plot_distribution: (bool) If true, plots the distribution of attributions
      and indicates the threshold point by a vertical line.

  Returns:
    (float) The threshold value.

  Raises:
    ValueError: if percentage is not in [0, 100].
  """
  if percentage < 0 or percentage > 100:
    raise ValueError('percentage must be in [0, 100]')
  
  # For percentage equal to 100, this should in theory return the lowest
  # value as the threshold. However, due to precision errors in numpy's cumsum,
  # the last value won't sum to 100%. Thus, in this special case, we force the
  # threshold to equal the min value.
  if percentage == 100:
    return np.min(attributions)

  flat_attributions = attributions.flatten()
  attribution_sum = np.sum(flat_attributions)
  
  # Sort the attributions from largest to smallest.
  sorted_attributions = np.sort(np.abs(flat_attributions))[::-1]

  # Compute a normalized cumulative sum, so that each attribution is mapped to
  # the percentage of the total sum that it and all values above it contribute.
  cum_sum = 100.0 * np.cumsum(sorted_attributions) / attribution_sum
  threshold_idx = np.where(cum_sum >= percentage)[0][0]
  threshold = sorted_attributions[threshold_idx]

  if plot_distribution:
    # Generate a plot of sorted intgrad scores.
    values_to_plot = np.where(cum_sum >= 95)[0][0]
    values_to_plot = max(values_to_plot, threshold_idx)
    plt.plot(np.arange(values_to_plot), sorted_attributions[:values_to_plot])
    plt.axvline(x=threshold_idx)
    plt.show()

  return threshold
    

def LinearTransform(attributions,
                    clip_above_percentile=99.9,
                    clip_below_percentile=70.0,
                    low=0.2,
                    plot_distribution=False):
  """Transform the attributions by a linear function.

  Transform the attributions so that the specified percentage of top attribution
  values are mapped to a linear space between `low` and 1.0.

  Args:
    attributions: (numpy.array) The provided attributions.
    percentage: (float) The percentage of top attribution values.
    low: (float) The low end of the linear space.

  Returns:
    (numpy.array) The linearly transformed attributions.

  Raises:
    ValueError: if percentage is not in [0, 100].
  """
  if clip_above_percentile < 0 or clip_above_percentile > 100:
    raise ValueError('clip_above_percentile must be in [0, 100]')
    
  if clip_below_percentile < 0 or clip_below_percentile > 100:
    raise ValueError('clip_below_percentile must be in [0, 100]')

  if low < 0 or low > 1:
    raise ValueError('low must be in [0, 1]')

  m = ComputeThresholdByTopPercentage(attributions,
                                      percentage=100-clip_above_percentile,
                                      plot_distribution=plot_distribution)
  e = ComputeThresholdByTopPercentage(attributions,
                                      percentage=100-clip_below_percentile,
                                      plot_distribution=plot_distribution)

  # Transform the attributions by a linear function f(x) = a*x + b such that
  # f(m) = 1.0 and f(e) = low. Derivation:
  #   a*m + b = 1, a*e + b = low  ==>  a = (1 - low) / (m - e)
  #                               ==>  b = low - (1 - low) * e / (m - e)
  #                               ==>  f(x) = (1 - low) (x - e) / (m - e) + low
  transformed = (1 - low) * (np.abs(attributions) - e) / (m - e) + low

  # Recover the original sign of the attributions.
  transformed *= np.sign(attributions)

  # Map values below low to 0.
  transformed *= (transformed >= low)
  
  # Clip values above and below.
  transformed = np.clip(transformed, 0.0, 1.0)
  return transformed


def Binarize(attributions, threshold=0.001):
  return attributions > threshold


def MorphologicalCleanup(attributions, structure=np.ones((4,4))):
  closed = ndimage.grey_closing(attributions, structure=structure)
  opened = ndimage.grey_opening(closed, structure=structure)
  
  return opened


def Outlines(attributions, percentage=90,
             connected_component_structure=np.ones((3,3)),
             plot_distribution=True):
  # Binarize the attributions mask if not already.
  attributions = Binarize(attributions)

  attributions = ndimage.binary_fill_holes(attributions)
  
  # Compute connected components of the transformed mask.
  connected_components, num_cc = ndimage.measurements.label(
      attributions, structure=connected_component_structure)

  # Go through each connected component and sum up the attributions of that
  # component.
  overall_sum = np.sum(attributions[connected_components > 0])
  component_sums = []
  for cc_idx in range(1, num_cc + 1):
    cc_mask = connected_components == cc_idx
    component_sum = np.sum(attributions[cc_mask])
    component_sums.append((component_sum, cc_mask))

  # Compute the percentage of top components to keep.
  sorted_sums_and_masks = sorted(
      component_sums, key=lambda x: x[0], reverse=True)
  sorted_sums = list(zip(*sorted_sums_and_masks))[0]
  cumulative_sorted_sums = np.cumsum(sorted_sums)
  cutoff_threshold = percentage * overall_sum / 100
  cutoff_idx = np.where(cumulative_sorted_sums >= cutoff_threshold)[0][0]

  if cutoff_idx > 2:
    cutoff_idx = 2
  
  # Turn on the kept components.
  border_mask = np.zeros_like(attributions)
  for i in range(cutoff_idx + 1):
    border_mask[sorted_sums_and_masks[i][1]] = 1

  if plot_distribution:
    plt.plot(np.arange(len(sorted_sums)), sorted_sums)
    plt.axvline(x=cutoff_idx)
    plt.show()

  # Hollow out the mask so that only the border is showing.
  eroded_mask = ndimage.binary_erosion(border_mask, iterations=1)
  border_mask[eroded_mask] = 0
  
  return border_mask


def Overlay(attributions, image):
  return np.clip(0.7 * image + 0.5 * attributions, 0, 255)




def pil_image(x):
  """Returns a PIL image created from the provided RGB array.

  Args:
    x: (numpy.array) RGB array of shape [height, width, 3] consisting of values
      in range 0-255.

  Returns:
    The PIL image.
  """
  x = np.uint8(x)
  return PIL.Image.fromarray(x)


def show_pil_image(pil_img):
  """Display the provided PIL image.

  Args:
    pil_img: (PIL.Image) The provided PIL image.
  """
  #f = StringIO()
  f = BytesIO()
  pil_img.save(f, 'png')
  display(Image(data=f.getvalue()))


G = [0, 255, 0]
R = [255, 0, 0]
def Visualize(attributions,
              image,
              positive_channel=G,
              negative_channel=R,
              polarity='positive',
              clip_above_percentile=99.9,
              clip_below_percentile=0,
              morphological_cleanup=False,
              structure=np.ones((3,3)),
              outlines=False,
              outlines_component_percentage=90,
              overlay=True,
              plot_distribution=False):
  
  if polarity == 'both':
    pos_attributions = Visualize(
        attributions, image, positive_channel=positive_channel,
        negative_channel=negative_channel, polarity='positive',
        clip_above_percentile=clip_above_percentile, clip_below_percentile=clip_below_percentile,
        morphological_cleanup=morphological_cleanup, outlines=outlines,
        outlines_component_percentage=outlines_component_percentage,
        overlay=False,
        plot_distribution=plot_distribution)
    
    neg_attributions = Visualize(
        attributions, image, positive_channel=positive_channel,
        negative_channel=negative_channel, polarity='negative',
        clip_above_percentile=clip_above_percentile, clip_below_percentile=clip_below_percentile,
        morphological_cleanup=morphological_cleanup, outlines=outlines,
        outlines_component_percentage=outlines_component_percentage,
        overlay=False,
        plot_distribution=plot_distribution)
    
    attributions = pos_attributions + neg_attributions
    
    if overlay:
      attributions = Overlay(attributions, image)
    
    return attributions
  elif polarity == 'positive':
    attributions = Polarity(attributions, polarity=polarity)
    channel = positive_channel
  elif polarity == 'negative':
    attributions = Polarity(attributions, polarity=polarity)
    attributions = np.abs(attributions)
    channel = negative_channel

  attributions = ConvertToGrayscale(attributions)
  
  attributions = LinearTransform(attributions,
                                 clip_above_percentile, clip_below_percentile,
                                 0.0,
                                 plot_distribution=plot_distribution)
  
  if morphological_cleanup:
    attributions = MorphologicalCleanup(attributions, structure=structure)
  if outlines:
    attributions = Outlines(attributions,
                            percentage=outlines_component_percentage,
                            plot_distribution=plot_distribution)
  
  # Convert to RGB space
  attributions = np.expand_dims(attributions, 2) * channel
  
  if overlay:
    attributions = Overlay(attributions, image)

  return attributions
  


In [0]:
import matplotlib.pyplot as plt
import numpy as np

In [0]:
labels = ['NG','OK']

#### Step 1. Load the model.

In [0]:
model = tf.keras.models.load_model(model_file)

#### Step 2. Be sure to complie it and add an optimizer.

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy')

#### Step 3. Wrap it with integrated gradients.

In [0]:
ig = integrated_gradients(model)

### Visualization

Step 1. Obtain a sample to explain.

In [0]:
sample_size = 1 # viualization할 이미지 개수, 현재 코드 상에서는 1
class_idx = 0 # NG 이면 0, OK 이면 1
iter = make_class_sapling(validation_fns, class_idx = class_idx , batch_size = sample_size)

images, labels = iter.get_next()
with tf.Session() as sess:
  images = sess.run(images)

In [0]:
plt.figure(figsize=(2,2))
plt.imshow(images[0])
plt.xticks([], [])
plt.yticks([], [])
plt.show()

In [0]:
# preprocess image
x = images

# preprocess reference as well
ref = np.zeros((224, 224, 3))
ref = np.expand_dims(ref, axis=0)
#ref = preprocess_input(ref)
ref = ref/255.0

Step 2. Predict

In [0]:
pred = model.predict(x)

In [0]:
predicted = np.argmax(pred)
print ("Predicted label:", labels[predicted])

Step 3. Explain with respect to the true label.

In [0]:
exp = ig.explain(x[0], reference=ref[0], outc=predicted)

In [0]:
plt.figure(figsize=(8,4))
plt.subplot(1, 2, 1)
plt.imshow(images[0])
plt.xticks([], [])
plt.yticks([], [])
plt.title("Original image")

th = max(np.abs(np.min(exp)), np.abs(np.max(exp)))
plt.subplot(1, 2, 2)
plt.imshow(np.sum(exp, axis=2), cmap="seismic", vmin=-1*th, vmax=th)
plt.xticks([], [])
plt.yticks([], [])
plt.title("Explanation")
plt.show()

In [0]:
# Clipping
print ('Clipping')
print ('The two graphs below show the top and bottom clipping on the attribution distribution curve.')

show_pil_image(pil_image(Visualize(
    exp, images[0] * 255,
    clip_above_percentile=95,
    clip_below_percentile=30,
    overlay=True,
    plot_distribution=True)))

In [0]:
# Morphological cleanup
print ('Clipping + Morphological cleanup')

show_pil_image(pil_image(Visualize(
    exp, images[0] * 255,
    clip_above_percentile=95,
    clip_below_percentile=30,
    morphological_cleanup=True,
    overlay=True)))

In [0]:
# Outlines
print ('Outlines')

show_pil_image(pil_image(Visualize(
    exp, images[0] * 255,
    clip_above_percentile=95,
    clip_below_percentile=30,
    morphological_cleanup=True,
    outlines=True,
    overlay=True)))

# SHAP

출처: https://slundberg.github.io/shap/notebooks/ImageNet%20VGG16%20Model%20with%20Keras.html

In [0]:
!pip install shap

In [0]:
import keras
from keras.preprocessing import image
import requests
from skimage.segmentation import slic
import matplotlib.pylab as plt
import numpy as np
import shap
from PIL import Image

In [0]:
from matplotlib.colors import LinearSegmentedColormap
colors = []
for l in np.linspace(1,0,100):
    colors.append((245/255,39/255,87/255,l))
for l in np.linspace(0,1,100):
    colors.append((24/255,196/255,93/255,l))
cm = LinearSegmentedColormap.from_list("shap", colors)

#### import model

In [0]:
feature_names=['NG','OK']

In [0]:
model = tf.keras.models.load_model(model_file)

### Visualization

In [0]:
sample_size = 1 # viualization할 이미지 개수, 현재 코드 상에서는 1
class_idx = 0 # NG 이면 0, OK 이면 1
iter = make_class_sapling(validation_fns, class_idx = class_idx, batch_size = sample_size)

images, labels = iter.get_next()
with tf.Session() as sess:
  imgs = sess.run(images)

In [0]:
img=Image.fromarray((imgs[0]*255).astype(np.uint8))
img_orig=imgs[0]

In [0]:
## 원래 코드인 segmentation 변환을 하면 결과가 너무 안 좋아서 image 자체를 사용해 봄
# segment the image so we don't have to explain every pixel
#segments_slic = slic(img, n_segments=30, compactness=30, sigma=3)
#segments_slic = slic(img, n_segments=40000, compactness=1000, sigma=0)
segments_slic = np.array(segimg)
plt.imshow(segments_slic);
plt.axis('off');

In [0]:
# define a function that depends on a binary mask representing if an image region is hidden
def mask_image(zs, segmentation, image, background=None):
    if background is None:
        background = image.mean((0,1))
    out = np.zeros((zs.shape[0], image.shape[0], image.shape[1], image.shape[2]))
    for i in range(zs.shape[0]):
        out[i,:,:,:] = image
        for j in range(zs.shape[1]):
            if zs[i,j] == 0:
                out[i][segmentation == j,:] = background
    return out
def f(z):
    return model.predict((mask_image(z, segments_slic, img_orig, 255)))
    #return model.predict(preprocess_input(mask_image(z, segments_slic, img_orig, 255)))

In [0]:
# use Kernel SHAP to explain the network's predictions
explainer = shap.KernelExplainer(f, np.zeros((1,50)))
shap_values = explainer.shap_values(np.ones((1,50)), nsamples=1000) # runs model 1000 times

In [0]:
# get the top predictions from the model
#preds = model.predict(preprocess_input(np.expand_dims(img_orig.copy(), axis=0)))
preds = model.predict(np.expand_dims(img_orig.copy(), axis=0))
top_preds = np.argsort(-preds)

In [0]:
def fill_segmentation(values, segmentation):
    out = np.zeros(segmentation.shape)
    for i in range(len(values)):
        out[segmentation == i] = values[i]
    return out

# plot our explanations
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,3))
inds = top_preds[0]
axes[0].imshow(img)
axes[0].axis('off')
max_val = np.max([np.max(np.abs(shap_values[i][:,:-1])) for i in range(len(shap_values))])
for i in range(2):
    m = fill_segmentation(shap_values[inds[i]][0], segments_slic)
    axes[i+1].set_title(feature_names[inds[i]])
    axes[i+1].imshow(img.convert('LA'), alpha=0.15)
    im = axes[i+1].imshow(m, cmap=cm, vmin=-max_val, vmax=max_val)
    axes[i+1].axis('off')
cb = fig.colorbar(im, ax=axes.ravel().tolist(), label="SHAP value", orientation="horizontal", aspect=60)
cb.outline.set_visible(False)
plt.show()

# LIME

출처: https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20Image%20Classification%20Keras.ipynb

In [0]:
!pip install lime

In [0]:
import os
import keras
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from skimage.segmentation import mark_boundaries
print('Notebook run using keras:', keras.__version__)

#### import model

In [0]:
model = tf.keras.models.load_model(model_file)

Let's see the top 5 prediction for some image

In [0]:
sample_size = 1 # viualization할 이미지 개수, 현재 코드 상에서는 1
class_idx = 0 # NG 이면 0, OK 이면 1
iter = make_class_sapling(validation_fns, class_idx = class_idx , batch_size = sample_size)

images, labels = iter.get_next()
with tf.Session() as sess:
  images = sess.run(images)

In [0]:
plt.imshow(images[0])
preds = model.predict(images)
print (preds)

#### Explanation
Now let's get an explanation

In [0]:
%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image


Parameters:	
- training_data – numpy 2d array
- training_labels – labels for training data. Not required, but may be used by discretizer.
- feature_names – list of names (strings) corresponding to the columns in the training data.
- categorical_features – list of indices (ints) corresponding to the categorical columns. Everything else will be considered continuous. Values in these columns MUST be integers.
- categorical_names – map from int to list of names, where categorical_names[x][y] represents the name of the yth value of column x.
- kernel_width – kernel width for the exponential kernel.
- None, defaults to sqrt (If) –
- verbose – if true, print local prediction values from linear model
- class_names – list of class names, ordered according to whatever the classifier is using. If not present, class names will be ‘0’, ‘1’, ...
- feature_selection – feature selection method. can be ‘forward_selection’, ‘lasso_path’, ‘none’ or ‘auto’. See function ‘explain_instance_with_data’ in lime_base.py for details on what each of the options does.
- discretize_continuous – if True, all non-categorical features will be discretized into quartiles.
- discretizer – only matters if discretize_continuous is True. Options are ‘quartile’, ‘decile’ or ‘entropy’

In [0]:
explainer = lime_image.LimeImageExplainer(verbose=True, feature_selection='auto')

hide_color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels. Here, we set it to 0 (in the representation used by inception model, 0 means gray)

```bash
explain_instance(image, classifier_fn, labels=(1, ), hide_color=None, top_labels=5, num_features=100000, num_samples=1000, batch_size=10, qs_kernel_size=4, distance_metric='cosine', model_regressor=None)
```

Generates explanations for a prediction.

First, we generate neighborhood data by randomly perturbing features from the instance (see __data_inverse). We then learn locally weighted linear models on this neighborhood data to explain each of the classes in an interpretable way (see lime_base.py).

Parameters:	
- data_row – 1d numpy array, corresponding to a row
- classifier_fn – classifier prediction probability function, which takes a numpy array and outputs prediction probabilities. For ScikitClassifiers , this is classifier.predict_proba.
- labels – iterable with labels to be explained.
- top_labels – if not None, ignore labels and produce explanations for the K labels with highest prediction probabilities, where K is this parameter.
- num_features – maximum number of features present in explanation
- num_samples – size of the neighborhood to learn the linear model
- distance_metric – the distance metric to use for weights.
- model_regressor – sklearn regressor to use in explanation. Defaults
- Ridge regression in LimeBase. Must have model_regressor.coef (to) –
- 'sample_weight' as a parameter to model_regressor.fit() (and) –
- qs_kernel_size – the size of the kernal to use for the quickshift segmentation


Returns:	
An Explanation object (see explanation.py) with the corresponding explanations.

In [0]:
%%time
# Hide color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels
#explanation = explainer.explain_instance(images[0], model.predict, top_labels=2, hide_color=0, num_features=1000, num_samples=2000)
explanation = explainer.explain_instance(images[0], model.predict, top_labels=2, hide_color=0, num_samples=3000)

Image classifiers are a bit slow. Notice that an explanation on my Surface Book dGPU took 1min 29s

#### Now let's see the explanation for the top class
We can see the top 1 superpixels that are most positive towards the class with the rest of the image hidden

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Or with the rest of the image present:

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

We can also see the 'pros and cons' (pros in green, cons in red)

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Or the pros and cons that have weight at least 0.1

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=1000, hide_rest=False, min_weight=0.1)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Let's see the explanation for the second highest prediction
Most positive towards wombat:

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[1], positive_only=False, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Pros and cons:

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[1], positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))